In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns

In [ ]:
df = pd.read_csv("~/Desktop/dataApiFast/used_car_sales.csv")

In [ ]:
zips = pd.read_csv("~/Desktop/dataApiFast/ZIP-COUNTY-FIPS_2017-06.csv")

In [ ]:
df.head()

In [ ]:
df.Make.unique()

In [ ]:
df.zipcode=df.zipcode.astype('str')
zips['ZIP']=zips['ZIP'].astype('str')

In [ ]:
df_with_state=pd.merge(df,zips[['ZIP','STATE']],left_on='zipcode',right_on='ZIP',how='inner')

In [ ]:
df_with_state=df_with_state[df_with_state.NumCylinders<17]

In [ ]:
df_with_state.NumCylinders.unique()

In [ ]:
df_with_state=df_with_state[df_with_state['Mileage']<300000]

In [ ]:
# Creating a heatmap using Seaborn
heatmap_data = df_with_state[['yearsold', 'pricesold', 'Mileage', 'Year']]
heatmap_data['yearsoldByMileage']=heatmap_data.yearsold*heatmap_data.Mileage
heatmap_data['Age']=heatmap_data['yearsold']-heatmap_data['Year']
heatmap_data['Age'][heatmap_data['Age']<0]=np.nan
heatmap_data['Age'][heatmap_data['Age']>150]=np.nan
heatmap_data['ageByMileage']=heatmap_data['Age']*heatmap_data['Mileage']



In [ ]:
heatmap = sns.heatmap(heatmap_data.corr(), annot=True, cmap='coolwarm', fmt=".2f")

#sns.scatterplot(data=heatmap_data, y='pricesold', x=heatmap_data['Age'])


# # Adding labels
# plt.title('Heatmap of Correlation Between Columns')
# plt.show()


In [ ]:
heatmap_data.corr()

In [ ]:
df_with_state=df_with_state[df_with_state['pricesold']<30000]

In [ ]:
make=pd.get_dummies(df_with_state['Make'])
drivetype=pd.get_dummies(df_with_state['DriveType'])
cylinders=pd.get_dummies(df_with_state['NumCylinders'])
body=pd.get_dummies(df_with_state['BodyType'])
state=pd.get_dummies(df_with_state['STATE'])



In [ ]:
df_with_state_and_dummies=pd.concat([df_with_state,make],axis=1)
df_with_state_and_dummies=pd.concat([df_with_state_and_dummies,drivetype],axis=1)
df_with_state_and_dummies=pd.concat([df_with_state_and_dummies,cylinders],axis=1)
df_with_state_and_dummies=pd.concat([df_with_state_and_dummies,body],axis=1)
df_with_state_and_dummies=pd.concat([df_with_state_and_dummies,state],axis=1)



In [ ]:
df_with_state_and_dummies['Age']=df_with_state_and_dummies['yearsold']-df_with_state_and_dummies['Year']
df_with_state_and_dummies=df_with_state_and_dummies[df_with_state_and_dummies['Age']<150]
df_with_state_and_dummies=df_with_state_and_dummies[df_with_state_and_dummies['Age']>-1]


In [ ]:
df_with_state_and_dummies.columns

In [ ]:
'Year','yearsold','ID','zipcode','ZIP','Engine'

In [ ]:
df_with_state_and_dummies.drop(columns=['Trim','Year','yearsold','ID','zipcode','ZIP','Engine','DriveType','NumCylinders','Make','Model','BodyType','STATE'],inplace=True)

In [ ]:
df_with_state_and_dummies.columns

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
X=df_with_state_and_dummies.drop(columns=['pricesold'])
X.select_dtypes(include='int')
X=X.astype('int')
y=df_with_state_and_dummies['pricesold']

In [ ]:
scaler=StandardScaler()



In [ ]:
reg = LinearRegression()
X_train,X_test,y_train,y_test=train_test_split(X.values,
y,
random_state=42,
test_size=.2)
scaler.fit(X_train)
X_train=scaler.transform(X_train)
X_test=scaler.transform(X_test)
pca=PCA(.95)
pca.fit(X_train)
X_train=pca.transform(X_train)
X_test=pca.transform(X_test)

In [ ]:
reg.fit(X_train,y_train)
y_pred=reg.predict(X_test)
mean_absolute_error(y_test,y_pred)

In [ ]:
plt.scatter(y_test,y_pred-y_test)